# Graph Visualization

<a href="https://githubtocolab.com/gleanerio/archetype/blob/master/networks/commons/notebooks/networkViz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.png" alt="Open in Colab"/></a>

## requirements.txt


In [ ]:
!pip install -q minio
!pip install -q kglab
!pip install -q sparqlwrapper
!pip install -q pygraphml
!pip install -q ipysigma

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)  ## remove pandas future warning
import kglab
from minio import Minio
from rdflib import Graph, plugin
import plotly.express as px
import pandas as pd
from urllib.request import urlopen
import os,json

from pygraphml import GraphMLParser
from pygraphml import Graph as GraphML 
import networkx as nx
from ipysigma import Sigma
from SPARQLWrapper import SPARQLWrapper, JSON
from hashlib import sha256

In [2]:

def hash_text(text):
  """
  Hashes the given text using SHA256 and returns a short hex digest.

  Args:
      text: The text to hash.

  Returns:
      A short hex digest of the hashed text.
  """
  return sha256(text.encode()).hexdigest()[:8]

In [3]:
## Remote SPARQL server
#@title
def get_sparql_dataframe(service, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    sparql = SPARQLWrapper(service)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)



In [8]:
rp1 = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX schema: <https://schema.org/>

SELECT  ?name ?url ?description ?s
WHERE {
    graph ?g {
        ?s schema:variableMeasured ?vm .
        ?vm ?p ?o
        OPTIONAL { ?vm schema:name ?name . }
        ?vm schema:url ?url . 
        OPTIONAL { ?vm schema:description ?description . }
    }
}
"""

ep = "http://0.0.0.0:7878/query"

df = get_sparql_dataframe(ep, rp1)


In [9]:
df.to_parquet('obisvars.parquet')
# df = pd.read_parquet('path/to/your/file.parquet')

In [16]:
df = df[df.applymap(lambda x: len(str(x)) > 0).all(axis=1)]


In [17]:
df.head(10)

,name,url,description,s
0,Length (fork length) of biological entity spec...,http://vocab.nerc.ac.uk/collection/P01/current...,Length (fork length) of biological entity spec...,https://obis.org/dataset/d6dc7351-aae6-4ed1-8f...
1,Length (fork length) of biological entity spec...,http://vocab.nerc.ac.uk/collection/P01/current...,Length (fork length) of biological entity spec...,https://obis.org/dataset/d6dc7351-aae6-4ed1-8f...
2,Length (fork length) of biological entity spec...,http://vocab.nerc.ac.uk/collection/P01/current...,Length (fork length) of biological entity spec...,https://obis.org/dataset/d6dc7351-aae6-4ed1-8f...
3,Length (fork length) of biological entity spec...,http://vocab.nerc.ac.uk/collection/P01/current...,Length (fork length) of biological entity spec...,https://obis.org/dataset/d6dc7351-aae6-4ed1-8f...
12,Sampling effort,http://vocab.nerc.ac.uk/collection/P02/current...,Sampling effort,https://obis.org/dataset/aecfd008-3399-4422-b7...
13,Sampling effort,http://vocab.nerc.ac.uk/collection/P02/current...,Sampling effort,https://obis.org/dataset/aecfd008-3399-4422-b7...
14,Sampling effort,http://vocab.nerc.ac.uk/collection/P02/current...,Sampling effort,https://obis.org/dataset/aecfd008-3399-4422-b7...
15,Sampling effort,http://vocab.nerc.ac.uk/collection/P02/current...,Sampling effort,https://obis.org/dataset/aecfd008-3399-4422-b7...
16,Lower sieve mesh size,http://vocab.nerc.ac.uk/collection/Q01/current...,Lower sieve mesh size,https://obis.org/dataset/55e369ca-ff48-4024-90...
17,Lower sieve mesh size,http://vocab.nerc.ac.uk/collection/Q01/current...,Lower sieve mesh size,https://obis.org/dataset/55e369ca-ff48-4024-90...


In [18]:
#  yeah, I get it...   don't iterate rows...  PR's welcome for this, being bad is too easy!  
# for index, row in nodes.iterrows():
#    g.add_node(row['Id'])
g = GraphML()
g.directed = False

for index, row in df.iterrows():
    n1 = g.add_node(row['s'])
    # n1['type'] = row['sType']
    
    n2 = g.add_node(row['name'])
    # n2['type'] = row['tType']

    e1 = g.add_edge(n1, n2)
    
fname = "./obisvars.graphml"
parser = GraphMLParser()
parser.write(g, fname)

In [19]:
# g = nx.les_miserables_graph()
g = nx.read_graphml("./obisvars.graphml")

In [20]:
Sigma(
    g,
    node_size=g.degree,
    default_edge_type='curve',
    node_border_color_from='node',
    node_metrics=['louvain'],
    node_color='louvain',
    start_layout=5,
    edge_size=lambda u, v: g.degree(u) + g.degree(v),
    edge_size_range=(0.5, 5),
    label_font='cursive',
    node_label_size=g.degree,
    label_density=0
)



Sigma(nx.MultiGraph with 2,230 nodes and 31,748 edges)